In [11]:
import numpy as np
import matplotlib.pyplot as plt
from casadi import *
from copy import *

ModuleNotFoundError: No module named 'casadi'

### System information: 

- nominal MPC (e.g. no noise or disturbances in process.)
- OCP: double integrator model
    - discrete-time system, linear model
    - wish to solve minimization s.t. u,x in the form:
        - $\min_{x, u} x_{N}^{\top}Q_{N}x_{N} + \sum_{k=0}^{N-1} x_{k}^{\top}Qx_{k}+u_{k}^{\top}Ru_{k}$
        - s.t.:
            - $|u_{k}| \leq 1 \quad \quad \forall_k = 0, 1,...,N-1$
            - $|y_{k}| \leq 1 \quad \quad \forall_k = 0, 1,...,N-1$
        - $x(i+1) = A x(i) + B u(i)$
        - $y(i) = C x(i)$
    - prediction horizon: $\quad 10$
    - terminal cost: $\qquad \quad x_{N}^{\top}Q_{N}x_{N}$
    - stage cost: $\qquad \qquad \sum_{k=0}^{N-1} x_{k}^{\top}Rx_{k}+u_{k}^{\top}Su_{k}$
- using off-the-shelf packages, CasADi functions to formulate the problem, Opti to solve

### Setting up the problem

Defining variables to formulate problem using helper CasADI functions

In [ ]:
x_states = 2 # total number of states
x = SX.sym('x', x_states) # symbol for states

u_controls = 1 # total number of control inputs
u = SX.sym('u', u_controls) # symbol for control inputs

y_outputs = 2 # total number of outputs

pred_hori = 10 # horizon

Defining matrices

In [ ]:
A, B, C = np.array([[1, 1], [0,1]]), np.array([[0.5], [1.0]]), np.array([[1, 0], [0, 1]])

Defining evolution equations

In [ ]:
f = Function('f', [x, u], [(A@x + B@u)]) # dynamics, e.g. state evolution, [name, input, output]
g = Function('g', [x], [(C@x)]) # output evolution 

In [ ]:
f_noise = Function('f_noise', [x, u], [((A*0.2)@x + B@u)]) # apply factor 0.2 onto A

Defining cost functions

In [ ]:
Q, R, S = np.eye(x_states), np.eye(u_controls), np.eye(x_states) # weight factors
stage_cost = Function('stage_cost', [x, u], [(x.T@Q@x) + (u.T@R@u)])
terminal_cost = Function('terminal_cost', [x], [x.T@S@x])

Defining constraints and initial values of the decision variables

In [ ]:
U_bound, Y_bound = 1., 1.
u_min, u_max = -U_bound * np.ones((u_controls, 1)), U_bound * np.ones((u_controls, 1))
y_min, y_max = -Y_bound * np.ones((y_outputs, 1)), Y_bound * np.ones((y_outputs, 1))

x_init, u_init, y_init = np.zeros(x_states), np.zeros(u_controls), np.zeros(y_outputs)

### OCP construction

In [ ]:
opti = Opti()

#### Open-loop construction

In [ ]:
# defining containers
X, U, Y = [0 for _ in range(pred_hori + 1)], [0 for _ in range(pred_hori)], [0 for _ in range(pred_hori + 1)] # tracking decision variables
noise_X, noise_U, noise_Y = deepcopy(X), deepcopy(U), deepcopy(Y)

In [ ]:
J_ol = 0 # initialize cost/objective
noise_J_ol = 0 # noise cost/objective

Add initial decision variables

In [ ]:
initial_state = np.array([-0.75, 0.75]).reshape(-1, 1)

In [ ]:
X[0], Y[0] = opti.parameter(x_states), opti.variable(y_outputs)
opti.set_value(X[0], initial_state) # afix known initial state e.g. k = 1
opti.subject_to(Y[0] == g(X[0])) # afix initial output decision variable e.g. k = 1

In [ ]:
noise_X[0], noise_Y[0] = opti.parameter(x_states), opti.variable(y_outputs)
opti.set_value(noise_X[0], initial_state) # noise
opti.subject_to(noise_Y[0] == g(noise_X[0])) # noise

Iteratively add the rest of the decision variables

In [ ]:
for k in range(pred_hori):    
    # Control input U
    U[k] = opti.variable(u_controls) # control input @ step k
    opti.set_initial(U[k], u_init) # initialize control input                                                            
    opti.subject_to(opti.bounded(u_min, U[k], u_max)) # constraints on control input
    
    # increment stage cost
    J_ol += stage_cost(X[k], U[k])

    # State X
    X[k + 1] = opti.variable(x_states) # state @ step k + 1
    opti.set_initial(X[k + 1], x_init) # initialize state at k + 1
    
    # Output Y
    Y[k + 1] = opti.variable(y_outputs) # output @ step k + 1
    opti.set_initial(Y[k + 1], y_init) # initialize output at k + 1
    opti.subject_to(opti.bounded(y_min, Y[k + 1], y_max)) # constraints on output 

    # constraints on dynamics and output equations
    opti.subject_to(X[k + 1] == f(X[k], U[k]))
    opti.subject_to(Y[k + 1] == g(X[k + 1]))

In [ ]:
# Noise decision variables
for k in range(pred_hori):    
    # Control input U
    noise_U[k] = opti.variable(u_controls) # noise    
    opti.set_initial(noise_U[k], u_init) # noise
    opti.subject_to(opti.bounded(u_min, noise_U[k], u_max)) # noise
    
    # increment stage cost
    noise_J_ol += stage_cost(noise_X[k], noise_U[k]) # noise

    # State X
    noise_X[k + 1] = opti.variable(x_states) # noise
    opti.set_initial(noise_X[k + 1], x_init) # noise
    
    # Output Y
    noise_Y[k + 1] = opti.variable(y_outputs) # output @ step k + 1
    opti.set_initial(noise_Y[k + 1], y_init) # initialize output at k + 1
    opti.subject_to(opti.bounded(y_min, noise_Y[k + 1], y_max)) # constraints on output 

    # constraints on dynamics and output equations
    opti.subject_to(noise_X[k + 1] == f_noise(noise_X[k], noise_U[k])) # noise 
    opti.subject_to(noise_Y[k + 1] == g(noise_X[k + 1])) # noise 

Terminate cost

In [ ]:
J_ol += terminal_cost(X[-1]) # indexed for the last state
noise_J_ol += terminal_cost(noise_X[-1]) 

Minimize the cost

In [ ]:
opti.minimize(J_ol)
opti.minimize(noise_J_ol)

### Solve OCP

#### [Ipopt](https://coin-or.github.io/Ipopt/): Input solver options, then solve.

In [ ]:
p_opts = {'verbose': False, 'expand': True, 'print_time': 1} # options taken from K.C. --> Ipopt options to print to console
s_opts = {'max_iter': 1000, 'print_level': 1, 'tol': 1e-6}

opti.solver('ipopt', p_opts, s_opts)

soln = opti.solve()

##### Open-loop solve

In [ ]:
# Extract array --> reshape to column --> Transpose
U_soln = np.array([soln.value(u) for u in U]).reshape(-1, u_controls).T
X_soln = np.array([soln.value(x) for x in X]).reshape(-1, x_states).T
Y_soln = np.array([soln.value(y) for y in Y]).reshape(-1, y_outputs).T

In [ ]:
noise_U_soln = np.array([soln.value(u) for u in noise_U]).reshape(-1, u_controls).T
noise_X_soln = np.array([soln.value(x) for x in noise_X]).reshape(-1, x_states).T
noise_Y_soln = np.array([soln.value(y) for y in noise_Y]).reshape(-1, y_outputs).T

### Plotting

No noise

In [ ]:
save_OL_X = deepcopy(X_soln)
save_OL_U = deepcopy(U_soln)

# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# X
for j in range(X_soln.shape[0]):
    ax1.plot(X_soln[j, :], '--', label=f'$x_{j}$')
if j > 0:
    ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
for i in range(U_soln.shape[0]):
    ax2.step(U_soln[i, :], '--', label=f'$u_{i}$')
if i > 0:
    ax2.legend()
ax2.set_ylabel('u, control input')
fig.suptitle('open-loop nominal OCP, no noise')
ax2.set_xlabel('k, steps')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()

Noise

In [ ]:
save_nOL_X = deepcopy(noise_X_soln)
save_nOL_U = deepcopy(noise_U_soln)

# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# X
for j in range(noise_X_soln.shape[0]):
    ax1.plot(noise_X_soln[j, :], '--', label=f'$x_{j}$')
if j > 0:
    ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
for i in range(noise_U_soln.shape[0]):
    ax2.step(noise_U_soln[i, :], '--', label=f'$u_{i}$')
if i > 0:
    ax2.legend()
ax2.set_ylabel('u, control input')
fig.suptitle('open-loop nominal OCP, with noise')
ax2.set_xlabel('k, steps')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()

#### Closed-loop construction

In [ ]:
sim_hori = 50 # simulation horizon

In [ ]:
# Extract array --> reshape to column --> Transpose
U_soln = np.array([soln.value(u) for u in U]).reshape(-1, u_controls).T
X_soln = np.array([soln.value(x) for x in X]).reshape(-1, x_states).T
Y_soln = np.array([soln.value(y) for y in Y]).reshape(-1, y_outputs).T

In [ ]:
noise_U_soln = np.array([soln.value(u) for u in noise_U]).reshape(-1, u_controls).T
noise_X_soln = np.array([soln.value(x) for x in noise_X]).reshape(-1, x_states).T
noise_Y_soln = np.array([soln.value(y) for y in noise_Y]).reshape(-1, y_outputs).T

In [ ]:
# defining containers
Xsim = np.zeros((x_states, sim_hori + 1)) # state evolution
Ysim = np.zeros((y_outputs, sim_hori + 1)) # output evolution
Usim = np.zeros((u_controls, sim_hori)) # input evolution
Jsim = np.zeros((1, sim_hori)) # cost evolution

In [ ]:
Xsim[:,0] = np.ravel(initial_state) # set the initial state
Ysim[:,0] = np.ravel(g(Xsim[:,0]).full())

Noise

In [ ]:
# defining containers
noise_Xsim = np.zeros((x_states, sim_hori + 1)) # state evolution
noise_Ysim = np.zeros((y_outputs, sim_hori + 1)) # output evolution
noise_Usim = np.zeros((u_controls, sim_hori)) # input evolution
noise_Jsim = np.zeros((1, sim_hori)) # cost evolution

noise_Xsim[:,0] = np.ravel(initial_state) # set the initial state
noise_Ysim[:,0] = np.ravel(g(Xsim[:,0]).full())

##### Closed-loop solve

In [ ]:
for k in range(sim_hori):
    opti.set_value(X[0], Xsim[:, k])
    sol = opti.solve()
    U_soln = np.array([sol.value(u) for u in U]).reshape(u_controls, -1)
    Y_soln = np.array([sol.value(y) for y in Y]).reshape(u_controls, -1)
    Usim[:, k] = U_soln[:, 0] # only interested in first optimal input in control

    Xsim[:, k + 1] = np.ravel(f(Xsim[:, k], Usim[:, k]).full()) # apply input to plant
    Ysim[:, k + 1] = np.ravel(g(Xsim[:, k + 1]).full())

In [ ]:
for k in range(sim_hori):
    opti.set_value(noise_X[0], noise_Xsim[:, k])
    sol = opti.solve()
    noise_U_soln = np.array([sol.value(u) for u in noise_U]).reshape(u_controls, -1)
    noise_Y_soln = np.array([sol.value(y) for y in noise_Y]).reshape(u_controls, -1)
    
    noise_Usim[:, k] = noise_U_soln[:, 0] # only interested in first optimal input in control

    noise_Xsim[:, k + 1] = np.ravel(f_noise(noise_Xsim[:, k], noise_Usim[:, k]).full()) # apply input to plant
    noise_Ysim[:, k + 1] = np.ravel(g(noise_Xsim[:, k + 1]).full())

In [ ]:
save_CL_X = deepcopy(Xsim)
save_CL_U = deepcopy(Usim)

# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# X
for i in range(Xsim.shape[0]):
    ax1.plot(Xsim[i, :], '--', label=f'$x_{i}$')
ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
ax2.step(np.arange(Usim.shape[1]),np.ravel(Usim), '--') 
ax2.set_ylabel('u, control input')

ax2.set_xlabel('k, steps')
fig.suptitle('Closed-loop nominal MPC, no noise')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()

In [ ]:
save_nCL_X = deepcopy(noise_Xsim)
save_nCL_U = deepcopy(noise_Usim)

# plot u, x on same x-axis (steps)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# X
for i in range(noise_Xsim.shape[0]):
    ax1.plot(noise_Xsim[i, :], '--', label=f'$x_{i}$')
ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
ax2.step(np.arange(noise_Usim.shape[1]),np.ravel(noise_Usim), '--') 
ax2.set_ylabel('u, control input')

ax2.set_xlabel('k, steps')
fig.suptitle('Closed-loop nominal MPC, with noise')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()

### Plotting match and mismatch

In [1]:
# no noise, match plant-model
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# X
for i in range(save_CL_X.shape[0]):
    ax1.plot(save_OL_X[i, :], '.-', label=f'$O-L x_{i}$')
    ax1.plot(save_CL_X[i, :], '--', label=f'$C-L x_{i}$')
ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
ax2.step(np.arange(save_OL_U.shape[1]),np.ravel(save_OL_U), '.-', label='O-L u') 
ax2.step(np.arange(save_CL_U.shape[1]),np.ravel(save_CL_U), '--', label='C-L u') 
ax2.set_ylabel('u, control input')

ax2.set_xlabel('k, steps')

fig.suptitle('plant-model MATCH')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()
# (CL_X OL_X) (CL_U OL_U)

In [ ]:
# noise, mismatch plantmodel
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# X
for i in range(save_nCL_X.shape[0]):
    ax1.plot(save_nOL_X[i, :], '.-', label=f'$O-L x_{i}$')
    ax1.plot(save_nCL_X[i, :], '--', label=f'$C-L x_{i}$')
ax1.legend()
ax1.set_ylabel('x, state')

# U --> step() as this is your controller knob
ax2.step(np.arange(save_nOL_U.shape[1]),np.ravel(save_nOL_U), '.-', label='O-L u') 
ax2.step(np.arange(save_nCL_U.shape[1]),np.ravel(save_nCL_U), '--', label='C-L u') 
ax2.set_ylabel('u, control input')

ax2.set_xlabel('k, steps')

fig.suptitle('plant-model MISMATCH')
ax1.set_xlim(-0.2, 10.2)
plt.tight_layout()
# (nCL_X nOL_X) (nCL_U nOL_U)